<a href="https://colab.research.google.com/github/PFCEC/unitrade/blob/main/%E6%95%99%E5%AD%B8/sample/unitrade_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 統一期貨unitradeAPI測試流程

# 事前準備

1.   完成開立統一期貨帳號，若未開戶請先進行開戶作業，開戶請見[統一期貨官網說明](https://www.pfcf.com.tw/eventweb/online/)
2.   完成新戶密碼開通，並取得電腦憑證，若無電腦憑證請至[憑證e總管](https://www.pscnet.com.tw/upload/pscnet/file/20230524094136409887.exe)申請
3. 向營業員申請統一API開通測試，並取得測試通知Mail
4. 將電腦憑證檔案上傳至Colab(點選左邊檔案icon，拖曳憑證檔案至網頁完成上傳)

  Tips:若找不到憑證，預設存放路徑為C:\User\使用者名稱\PSCCA\PSC_您的ID_到期日.pfx

---


# 注意事項

*   該項API申請會同步開通C#、VBA、Python版本(本份文件以Python版本撰寫)
*   客戶需自行操作Colab並理解程式碼，不可委由他人或營業員操作
*   完整Python API操作說明請點[unitradeAPI官方網站](https://pfcec.github.io/unitrade/)
*   須完成測試委託一筆，顯示「委託成功」即可，不用成交(測試環境也不會成交)
*   「委託成功」完成後，請通知您的營業員，後續等待正式統一API開通信即可
*   本份文件預設連線為「測試伺服器」若自行更改連線伺服器，須自行承擔交易內容


# 輸入您的帳戶資料

In [ ]:
# 按 Shift + Enter 執行程式碼 或 按左邊按鈕執行程式碼

#============================================================#
# 請輸入帳號(含分公司碼，合計共11碼) (勿刪除"")

ACT_KEY = ""

# 請輸入交易密碼(同登入密碼)

PSW_KEY = ""

# 請輸入憑證檔名

CA_NAME_KEY = ""

# 請輸入憑證密碼(非交易密碼，若未設定密碼，則刪除中文字即可)

CA_PSW_KEY = ""
#============================================================#


# 安裝套件

In [ ]:
!pip install pandas
!pip install unitrade

# 初始化設定

In [ ]:
import unitrade
import pandas as pd
from unitrade.unitrade import *
from typing import List
from IPython.display import display,clear_output

#初始化API
api = Unitrade()

#匯入顯示資料套件
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

#初始資料
reply_data = [ ['','','','','','','','','','','','','','','','','','','','','']]
reply_dd = pd.DataFrame(reply_data)
reply_dd.columns=['分公司','帳號','網路流水序號','委託時間','委託書號','子帳',
            '商品類別','商品代碼','買賣別','價格別','委託價格','委託數量','未成交數量',
            '成交數量','刪除數量','下單方式','開倉別','異動時間','委託狀態','委託狀態碼','下單序號']
match_data = [ ['','','','','','','','','','','','','','','','']]
match_dd = pd.DataFrame(match_data)
match_dd.columns=['分公司','帳號','網路流水序號','成交時間','委託書號','子帳',
            '商品類別','商品代碼','買賣別','成交價格','成交口數','成交序號',
            '成交價1','成交價2','備註','異動時間']
#訂閱錯誤事件回報
api.on_error = lambda error: print(f'api錯誤 {error}')

#訂閱回報並設定接收欄位
def on_reply(orderreply: DOrderReply):
    i=len(reply_dd)+1
    reply_dd.loc[i, '分公司'] = orderreply.brokerid
    reply_dd.loc[i, '帳號'] = orderreply.investoracno
    reply_dd.loc[i, '網路流水序號'] = orderreply.networkid
    reply_dd.loc[i, '委託時間'] = orderreply.ordertime
    reply_dd.loc[i, '委託書號'] = orderreply.orderno
    reply_dd.loc[i, '子帳'] = orderreply.subact
    reply_dd.loc[i, '商品類別'] = orderreply.productkind
    reply_dd.loc[i, '商品代碼'] = orderreply.productid
    reply_dd.loc[i, '買賣別'] = orderreply.bs
    reply_dd.loc[i, '價格別'] = orderreply.ordertype
    reply_dd.loc[i, '委託價格'] = orderreply.price
    reply_dd.loc[i, '委託數量'] = orderreply.orderqty
    reply_dd.loc[i, '未成交數量'] = orderreply.nomatchqty
    reply_dd.loc[i, '成交數量'] = orderreply.matchqty
    reply_dd.loc[i, '刪除數量'] = orderreply.delqty
    reply_dd.loc[i, '下單方式'] = orderreply.ordercondition
    reply_dd.loc[i, '開倉別'] = orderreply.opencloseflag
    reply_dd.loc[i, '異動時間'] = orderreply.mdate
    reply_dd.loc[i, '委託狀態'] = orderreply.orderstatus
    reply_dd.loc[i, '委託狀態碼'] = orderreply.statuscode
    reply_dd.loc[i, '開倉別'] = orderreply.opencloseflag
    reply_dd.loc[i, '下單序號'] = orderreply.seq
    clear_output(wait=True)
    display(reply_dd)
    display(match_dd)
def on_match(matchreply: DMatchReply):
    i=len(reply_dd)+1
    match_dd.loc[i, '分公司'] = matchreply.brokerid
    match_dd.loc[i, '帳號'] = matchreply.investoracno
    match_dd.loc[i, '網路流水序號'] = matchreply.networkid
    match_dd.loc[i, '成交時間'] = matchreply.matchtime
    match_dd.loc[i, '委託書號'] = matchreply.orderno
    match_dd.loc[i, '子帳'] = matchreply.subact
    match_dd.loc[i, '商品類別'] = matchreply.productkind
    match_dd.loc[i, '商品代碼'] = matchreply.productid
    match_dd.loc[i, '買賣別'] = matchreply.bs
    match_dd.loc[i, '成交價格'] = matchreply.matchprice
    match_dd.loc[i, '成交口數'] = matchreply.matchqty
    match_dd.loc[i, '成交序號'] = matchreply.matchseq
    match_dd.loc[i, '成交價1'] = matchreply.matchpricefoot1
    match_dd.loc[i, '成交價2'] = matchreply.matchpricefoot2
    match_dd.loc[i, '備註'] = matchreply.note
    match_dd.loc[i, '異動時間'] = matchreply.mdate
    clear_output(wait=True)
    display(reply_dd)
    display(match_dd)
api.dtrade.on_reply=on_reply
api.dtrade.on_match=on_match
print('訂閱完畢')

# 期貨測試下單

In [ ]:
#系統登入

# 請勿更動連結網址，確保API在測試環境運行!!!
#======================================================================================================================#
loginresponse = api.login(
    "https://test167.pfctrade.com", ACT_KEY, PSW_KEY, CA_NAME_KEY, CA_PSW_KEY) #url,帳號,密碼,憑證檔名,憑證密碼

actno = api.get_accounts()[0] #取得第一個交易帳號 (此為免公司碼，7碼之帳號)
#======================================================================================================================#
# 自行更動網址，請自負交易風險!!!

#顯示登入結果
print(loginresponse)
print(f"登入旗標{api.login_status_flag}")
print(f"交易帳號{actno}")

#下單
order = DOrderObject() #初始化委託
order.actno = actno #帳號
order.note = "ordertest" #備註
order.subactno = "" #子帳 (若無子帳則空白)
order.productid = "TXFI5" #商品代號 #預設為大台25年9月 (若契約已到期，請更改代碼，代碼請見統eVIP全球板)
order.bs = "B" #買賣別B/S
order.ordertype = "M" #L:限價 M:市價 P:範圍市價
order.price = 0 #委託價格 (若市價則設定0，限價請指定價格)
order.orderqty = 1 #數量
order.ordercondition = "R" #委託種類 I:IOC R:ROD F:FOK
order.opencloseflag = "" #開倉別 str 0:新倉 1:平倉 空白:自動
order.dtrade="N" # Y:當沖 N:非當沖
orderresponse=api.dtrade.order(order) #委託送出
print(f" 是否成功 {orderresponse.issend} 下單序號 {orderresponse.seq} 錯誤代碼 {orderresponse.errorcode}"
   f" 錯誤訊息 {orderresponse.errormsg}")
time.sleep(5) #等待Colab運行5秒，讓回報順利回來

#系統登出
api.logout()
print('登出完畢')
print(f"登入旗標{api.login_status_flag}")

# 首次登入需較久時間，請耐心等候
# 若出現「api錯誤 faccount_on_error 連線失敗」會自動重連，請勿強制停止程式
# 顯示「委託成功」即完成丟單測試
# 請通知您的營業員，完成後續正式API開通